In [10]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from statsmodels.stats.proportion import proportion_confint
from itertools import product

## Importing

In [11]:
parts_path = Path(r'E:\after_session_03_22\data\part_data')
os.listdir(parts_path)

['part1_dm_combined_18_95_J01.pkl',
 'part2_dm_combined_18_95_J01.pkl',
 'part3_dm_combined_18_95_J01.pkl',
 'part4_dm_combined_18_95_J01.pkl',
 'part5_dm_combined_18_95_J01.pkl',
 'part6_dm_combined_18_95_J01.pkl',
 'part7_dm_combined_18_95_J01.pkl',
 'part8_dm_combined_18_95_J01.pkl',
 'without prov_mod']

## Calculating

In [18]:
# to_groupby = ['gender', 'age_cat', 'cat', 'member']
to_groupby = 'member'

first_iter = True
for part_loc in os.listdir(parts_path):
    # Logging
    print('Processing:  ', part_loc)

    # Importing part
    part_loc = parts_path / part_loc
    part = pd.read_pickle(part_loc)

    # Filtering for ab drugs
    part = part.loc[part['ab']]

    # Building ab_class column
    part['ab_class'] = part['atc'].str[:4]
    # groupby
    if first_iter:
        df = part.groupby(to_groupby)['ab_class'].value_counts().to_frame().unstack(-1)
    first_iter = False
    df = pd.concat([
        df, part.groupby(to_groupby)['ab_class'].value_counts().to_frame().unstack(-1)
    ], axis=0)


Processing:   part1_dm_combined_18_95_J01.pkl
Processing:   part2_dm_combined_18_95_J01.pkl
Processing:   part3_dm_combined_18_95_J01.pkl
Processing:   part4_dm_combined_18_95_J01.pkl
Processing:   part5_dm_combined_18_95_J01.pkl
Processing:   part6_dm_combined_18_95_J01.pkl
Processing:   part7_dm_combined_18_95_J01.pkl
Processing:   part8_dm_combined_18_95_J01.pkl


In [25]:
# Deleting duplicates
df = df.groupby(df.index).last()

In [ ]:
df

In [27]:
df.index.get_level_values('member').nunique()

1394977

## Adding Data From person_df_J01

In [ ]:
person_df_J01 = pd.read_pickle(r'E:\after_session_03_22\data\one_df\person_df_J01.pkl')
person_df_J01

In [28]:
# Merging
cols_to_merge = ['gender', 'fund', 'age_cat', 'cat', 'province']

final = pd.merge(
    df.loc[:, 'ab_class'],
    person_df_J01[cols_to_merge],
    how='left',
    on='member'
)

In [ ]:
final

## Saving

In [31]:
# Saving ab_class_member data
# final.to_pickle(r'E:\after_session_03_22\data\drug_class\ab_drug_class_numbers.pkl')